In [45]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round

# Initialize a SparkSession
spark = SparkSession.builder \
        .appName("MySparkApp") \
        .getOrCreate()

In [46]:
# Import the necessary type as classes 
from pyspark.sql.types import (StructType, 
                               StructField, 
                               IntegerType, StringType, ArrayType)

# Construct the schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("scores", ArrayType(IntegerType()), True)
])

# Create the data
data = [(1, "Alice", [100, 80, 75]),
        (2, "Bob", [90, 88, 91]),
        (3, "Charlie", [85, 90, 80])]

# Set the schema
df = spark.createDataFrame(data, schema=schema)
df.show()


+---+-------+-------------+
| id|   name|       scores|
+---+-------+-------------+
|  1|  Alice|[100, 80, 75]|
|  2|    Bob| [90, 88, 91]|
|  3|Charlie| [85, 90, 80]|
+---+-------+-------------+



In [47]:
# Create DataFrame
salaries_df = spark.read.csv("datasets/salaries.csv", header=True, inferSchema=True)
salaries_df.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2020|              EN|             FT| Azure Data Engineer|100000|            USD|       100000|                MU|           0|              MU|           S|
|     2020|              EN|             CT|  Staff Data Analyst| 60000|            CAD|        44753|                CA|          50|              CA|           L|
|     2020|              SE|             FT|Staff Data Scientist|164000|            USD|       164000|                US|          50|              US|           M|
|     2020

In [48]:
salaries_df.printSchema()

root
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [49]:
row_count = salaries_df.count()
print(f"Number of rows: {row_count}")

Number of rows: 37234


In [50]:
salaries_df.groupBy('experience_level') \
    .agg(
       round(avg('salary'), 2).alias('average_salary'),
       round(avg('salary_in_usd'), 2).alias('average_salary_usd')
    ) \
    .show()

+----------------+--------------+------------------+
|experience_level|average_salary|average_salary_usd|
+----------------+--------------+------------------+
|              EX|     205993.21|         198208.34|
|              MI|     155797.43|         144187.63|
|              EN|     121039.41|         107310.08|
|              SE|     176935.42|         174433.86|
+----------------+--------------+------------------+



In [51]:
salaries_df.groupBy("company_size").agg({"salary_in_usd": "avg"}).show()

+------------+------------------+
|company_size|avg(salary_in_usd)|
+------------+------------------+
|           L|147706.87870434183|
|           M|161471.45763426356|
|           S|          88036.29|
+------------+------------------+



In [52]:
# Average salary for entry level in Canada
CA_jobs = salaries_df.filter(salaries_df['company_location'] == "CA").filter(salaries_df['experience_level'] == "EN").groupBy().avg("salary_in_usd")

# Show the result
CA_jobs.show()

+------------------+
|avg(salary_in_usd)|
+------------------+
| 97330.53932584269|
+------------------+



In [53]:
# Drop null values
salaries_df_cleaned = salaries_df.na.drop()

In [54]:
row_count = salaries_df.count()
row_count

37234

In [55]:
salaries_df.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2020|              EN|             FT| Azure Data Engineer|100000|            USD|       100000|                MU|           0|              MU|           S|
|     2020|              EN|             CT|  Staff Data Analyst| 60000|            CAD|        44753|                CA|          50|              CA|           L|
|     2020|              SE|             FT|Staff Data Scientist|164000|            USD|       164000|                US|          50|              US|           M|
|     2020

In [56]:
salaries_df_v02 = salaries_df.withColumn('weekly_salary', salaries_df['salary'] / 52)
salaries_df_v02.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+------------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|     weekly_salary|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+------------------+
|     2020|              EN|             FT| Azure Data Engineer|100000|            USD|       100000|                MU|           0|              MU|           S| 1923.076923076923|
|     2020|              EN|             CT|  Staff Data Analyst| 60000|            CAD|        44753|                CA|          50|              CA|           L|1153.8461538461538|
|     2020|              SE|             FT|Staff Data Scientist|164000|        